<a href="https://colab.research.google.com/github/KIMMOCEE/Section1_Project/blob/main/ai12_sc21x_%EA%B9%80%ED%98%81%EC%A7%84_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확인하세요!

### 제출 전에 모든 코드가 순차적으로 잘 작동하는지 확인하세요. 
- **커널을 다시 시작**한 다음(메뉴바에서 Kernel$\rightarrow$Restart 선택) **모든 셀을 실행**해 보세요(메뉴바에서 Cell$\rightarrow$Run All 선택).

### 지정 셀의 지정 영역에서만 코드를 작성했는지 확인하세요.
- `YOUR CODE HERE`이 적혀 있는 셀에 대한 코드만 채점에 반영됩니다.
- `YOUR CODE HERE` ~ `YOUR CODE ENDS HERE` 사이에 본인의 코드를 작성하세요.
- 지정 셀 이외에 본인이 코드 실행 결과를 확인하기 위한 용도로 셀을 추가해도 됩니다.
    - **제출 시 지정 셀에서는 정답 이외에 결과 확인을 위한 코드 등은 제거하고 제출해 주세요!**
    - **기존 셀을 삭제하지 마세요! 채점에 불이익이 있을 수 있습니다.**

### 제출 방법
- 시간 내에 문제를 풀고, 본인의 github repo에 push 하세요.
    - **디렉토리 구조가 변하면 안 됩니다.** `codestates/ds-sc2xx`와 디렉토리 구조가 동일해야 합니다. 답안 파일의 이름을 변경하지 않았는지, 기존에 없던 폴더가 추가되지 않았는지 등을 확인하세요.
    - **코드스테이츠 가입 시 연동한 github 계정을 사용해야 합니다.**
    - **branch를 변경하여 작업하지 마세요.**
        - `master` branch에의 제출 사항에 대해서만 평가합니다.
        - 본인의 github repo `master` branch에 제출이 잘 되어 있는지 확인하세요.

---

# SC21x
 
## 선형모델 (Linear Models)

In [ ]:
# # Google Colab에서 하시는 경우 해당 셀을 먼저 실행해주시기 바랍니다.
import sys
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*

# Part 1 - 분류 (Classification): 헌혈 여부 예측하기 🚑
Part 1에선 대만의 이동식 헌혈 차량 헌혈자들의 정보를 담은 데이터셋을 다룹니다. 대만의 수혈 서비스 센터 (The Blood Transfusion Service Center)는 이동식 차량으로 대학교들을 돌면서 헌혈 행사를 진행, 수혈을 위한 혈액을 기부 받습니다.

Part 1의 목표는 각 헌혈자의 정보를 활용해서 **헌혈자가 2007년 3월에 헌혈을 했는지 여부**를 예측하는 것입니다. 

헌혈 여부 및 공급 요구를 추적하고 예측하는 좋은 데이터 기반 시스템은 공급망 전체를 개선하여 더 많은 환자가 필요한 수혈을 받도록 할 수 있습니다.

In [1]:
# 분석에 필요한 라이브러리 및 데이터셋을 불러옵니다
import pandas as pd

donors = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')
assert donors.shape == (748,5)  # 데이터셋이 잘 로드 됐는지 확인하기 위해서 assert를 사용해 검증합니다.

# 칼럼의 이름을 이해하기 쉽도록 변경합니다.
donors = donors.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [2]:
donors.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


## 칼럼 설명:
- **months_since_last_donation**: 마지막으로 헌혈을 한 후에 지난 개월수
- **number_of_donations**: 헌혈 횟수
- **total_volume_donated**: 기부한 혈액 총량
- **months_since_first_donation**: 첫 헌혈을 한 후에 지난 개월수
- **made_donation_in_march_2007**: 2007년 헌혈을 했는지 여부 (target)

데이터를 본다면 약 3/4에 해당하는 과반수가 2007년 3월에 헌혈을 하지 않았다는 것을 알 수 있습니다.  
아래는 Baseline 모델의 정확도 점수 (accuracy score)를 나타낸 것입니다.

In [3]:
donors['made_donation_in_march_2007'].value_counts(normalize=True)

0    0.762032
1    0.237968
Name: made_donation_in_march_2007, dtype: float64

## 1.1 데이터를 feature (X), label(y)로 분할하고 데이터를 train/test 셋으로 나누는 `divide_donors_data_and_split`함수를 작성하세요 (scikit-learn 활용).
- donors dataframe을 독립변수 dataframe과 종속변수 series로 나눠 주세요
  - 종속변수 외 나머지 변수는 모두 독립변수로 간주하세요
- sklearn train_test_split 함수를 사용해 Train / Test 비율을 3:1로 나눠 주세요. random_state는 42를 사용하세요.

In [4]:
from sklearn.model_selection import train_test_split


def divide_donors_data_and_split(donors):
  # YOUR CODE HERE
  feature = ['months_since_last_donation', 'number_of_donations', 'total_volume_donated', 'months_since_first_donation']
  label ='made_donation_in_march_2007'
  X = donors[feature]
  y = donors[label]
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, test_size=0.25, random_state=42)
  # YOUR CODE ENDS HERE
  return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = divide_donors_data_and_split(donors)

## 1.2 Logistic regression 모델을 학습 (fit)하고 학습된 모델과 test accuracy를 반환하는 `fit_logistic_regression`함수를 작성하세요.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fit_logistic_regression(X_train, X_test, y_train, y_test):
  logistic = LogisticRegression()
  test_accuracy = 0
  # YOUR CODE HERE
  logistic.fit(X_train, y_train)
  test_accuracy = logistic.score(X_test, y_test)
  # YOUR CODE ENDS HERE
  return logistic, test_accuracy

In [7]:
logistic, test_accuracy = fit_logistic_regression(X_train, X_test, y_train, y_test)

## 1.3 분류 평가 지표 (classification metric) 확인: 정확도 (accuracy)
- 아래 셀을 실행하여 테스트셋을 사용했을 때 분류 모델의 정확도(accuracy score)를 확인해 보세요.

- 모델의 성능이 베이스라인보다 안좋더라도 걱정하지 마세요. 
  - accuracy 대신 recall을 평가 지표 (metric)로 사용할 경우에 우리가 만든 모델이 베이스라인을 뛰어넘을 수 있습니다. 알맞은 평가 지표를 고르고 해석하는 법은 앞으로 계속 다룰 예정입니다.

In [8]:
print("test accuracy: ", test_accuracy)

test accuracy:  0.7540106951871658


---

# 수고하셨습니다!